In [ ]:
import re # check what is it (regular expression)
import pandas as pd

In [ ]:
path_file = ""
f = open(path_file,'r',encoding='utf-8')

In [ ]:
data = f.read() # read content in string format.

In [ ]:
print(type(data))

In [ ]:
print(data)
# convert this into Pandas Data-Frame.

In [ ]:
date = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s\w\w'
messages = re.split(date, data)[1:]
messages


In [ ]:
# Extract Date and time
dates = re.findall(date, data)
dates

In [ ]:
len(dates)

In [ ]:
len(messages)

In [ ]:
# delete text which has empty string
del messages[27200:27311]

In [ ]:

len(messages)

In [ ]:
dates[2]

In [ ]:
# create DataFrame
df = pd.DataFrame({'user_message':messages, 'messages_date':dates})
# convert message_date type
df['messages_date'] = pd.to_datetime(df['messages_date'], format='%m/%d/%y, %I:%M %p')

df.rename(columns={'messages_date':'date'}, inplace=True)

df.head()

In [ ]:
df.shape

In [ ]:
# Now, split user_name from user_message.[separate users and messages]
users = []
message = []
for msg in df['user_message']:
    entry = re.split('([\w\W]+?):\s', msg)
    if entry[1:]:#user_name
        users.append(entry[1])
        message.append(entry[2])
    else:
        users.append('group_notification')
        message.append(entry[0])
df['user'] = users
df['message'] = message
df.drop(columns=['user_message'], inplace=True)

df.head()

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name()
df['day'] = df['date'].dt.day


In [ ]:
df.head()

In [ ]:
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

In [ ]:
df.head()

# DataFrame Ready

In [ ]:
# user_list = df['user'].unique().tolist()
# print(user_list)

In [ ]:
# df['user'].value_counts()

In [ ]:
# delete_user_list = ['coordinate (h) ', 'coordinate (k) ', 'Home']# Personal Delete
# user_list.remove('Home')# Personal Delete
# user_list

In [ ]:
# words = []
# for msg in df['message']:
#     words.extend(msg.split())

In [ ]:

# len(words)

In [ ]:
# new_df = df[df['user'] == 'user_name']

In [ ]:
# len(new_df[new_df['message'] == '<Media omitted>'])

In [ ]:
# pip install urlextract

In [ ]:
# Extract all links in chat
# from urlextract import URLExtract

In [ ]:
# extractor = URLExtract()
# urls = extractor.find_urls('www.abc.org taos www.abamnad.com snd https://ramlasksan.in')
# urls

In [ ]:
# links = []
# for msg in df['message']:
#     links.extend(extractor.find_urls(msg))
# links

In [ ]:
# x = df['user'].value_counts().head()

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# name = x.index
# count = x.values

In [ ]:
# plt.bar(name, count)
# plt.xticks(rotation = 'vertical')
# plt.show()

In [ ]:
# round((df['user'].value_counts()/df.shape[0])*100,2).reset_index().rename(columns={'index':'

In [ ]:
words = []
for msg in df['message']:
    words.extend(msg.split())
words

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
# remove - group notifications messages 
# remove - media omitted
# remove - stop words

In [ ]:
# df[df['user'] == 'group_notification']

In [ ]:
df[df['user'] != 'group_notification']

In [ ]:
df[df['message'] == '<Media omitted>']

In [ ]:
f = open('stop_hinglish.txt', 'r')
stop_words = f.read()
print(stop_words)

In [ ]:
temp = df[df['user'] != 'group_notification']
temp = temp[temp['message'] != '<Media omitted>']

In [ ]:
words = []
for msg in temp['message']:
    for word in msg.lower().split():
        if word not in stop_words:
            words.append(word)
words

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
pip install emoji
import emoji

In [ ]:
emojis = []
for msg in df['message']:
    emojis.extend([c for c in msg if c in emoji.UNICODE_EMOJI['en']])

In [ ]:
emojis_df = pd.DataFrame(Counter(emojis).most_common(20))

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.pie(emojis_df[1].head(), labels=emojis_df[0].head(), autopct='%0.2f')
plt.show()

In [ ]:
df['month_num'] = df['date'].dt.month

In [ ]:
timeline = df.groupby(['year', 'month_num','month']).count()['message'].reset_index()

In [ ]:
timeline

In [ ]:
# merge the month and year.
time = []
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i]  + ',' + str(timeline['year'][i]))

In [ ]:
timeline['time'] = time

In [ ]:
timeline

In [ ]:
plt.plot(timeline['time'], timeline['message'])
plt.xticks(rotation='vertical')
plt.show();

In [ ]:
# Date timeline
df.head()

In [ ]:
df['only_date'] = df['date'].dt.date

In [ ]:
daily_timeline = df.groupby('only_date').count()['message'].reset_index()

In [ ]:
# daily_timeline

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(daily_timeline['only_date'], daily_timeline['message'])
plt.xticks(rotation = 'vertical')
plt.show();

In [ ]:
# How and at what the person get active in group.

In [ ]:
df['day_name'] = df['date'].dt.day_name()

In [ ]:
df['day_name'].value_counts()

In [ ]:
# Create Heatmap to check active timing of the user.
df.head()

In [ ]:
# humko timing ka range create krna pdega
period = []
for hour in df[['day_name', 'hour']]['hour']:
    if hour == 23:
        period.append(str(hour) + '-' + str('00'))
    elif hour < 9:
        period.append(str('0') + str(hour) +  '-' + str('0') + str(hour+1))
    elif hour == 9:
           period.append(str('0') + str(hour) +  '-' + str(hour+1))
    else:
        period.append(str(hour) + '-' + str(hour+1))

In [ ]:

df['period'] = period

In [ ]:
# df.sort_values('period' , inplace=True)

In [ ]:

import seaborn as sns

In [ ]:
plt.figure(figsize=(20,6))
sns.heatmap(df.pivot_table(index='day_name', columns='period', values='message', aggfunc='count').fillna(0))
plt.yticks(rotation='horizontal')
plt.show()

In [ ]:
df.head()